 ## 🏥 Geração de Dados Médicos Fictícios com Faker e Random

Este notebook demonstra como criar um **conjunto de dados médicos fictícios**
utilizando as bibliotecas **Faker** e **random**, ideal para testes,
análises exploratórias e demonstrações sem expor informações reais.

### 📌 Funcionalidades do código:
- **Dados Pessoais**: Geração de nomes realistas em português, idade, altura, peso e gênero.
- **Histórico Médico**: Criação de registros médicos probabilísticos, variando com a idade.
- **Consultas**: Datas de atendimento nos últimos 2 anos.
- **Avaliação do Serviço**: Notas aleatórias de 0 a 10.
- **Sintomas e Tratamentos**: Seleção de sintomas (ex.: *Resfriado*, *COVID-19*, *Tuberculose*),
  com resultados e tratamentos ajustados conforme a gravidade.

### 🔧 Bibliotecas Utilizadas:
- **pandas / numpy**: Estruturação e manipulação dos dados gerados.
- **faker**: Geração de nomes, datas e textos realistas.
- **random**: Sorteio de números, categorias e probabilidades.

> 💡 **Uso prático**: Ideal para criar datasets fictícios para treinamento de modelos,
protótipos de dashboards, testes de sistemas de saúde ou estudos acadêmicos.

In [13]:
import pandas as pd
import numpy as np
from faker import Faker
import random

fake = Faker('pt_BR') # Usando Faker com localização brasileira

def generate_medical_data(num_records):
    data = []
    for _ in range(num_records):
        name = fake.name()
        age = random.randint(18, 80)
        height = round(random.uniform(1.50, 1.90), 2)
        weight = round(random.uniform(50, 100), 2)
        gender = random.choice(['Masculino', 'Feminino', 'Outro', np.nan])

        # Histórico médico mais provável com a idade
        medical_history = fake.sentence() if random.random() < (age / 100.0) + 0.1 else np.nan # Aumenta a probabilidade com a idade

        consultation_date = fake.date_between(start_date='-2y', end_date='today')
        service_rating = random.randint(0, 10) # Adicionando classificação do atendimento

        # Adiciona Sintomas e relaciona com Resultado e Tratamento
        symptoms = random.choice(['Resfriado', 'Infecção de Garganta', 'COVID-19', 'Tuberculose'])
        outcome = 'Melhorou' # Resultado padrão
        treatment = 'Medicação Comum (analgésicos/antitérmicos/antigripais.)' # Tratamento padrão
        severity = np.nan # Gravidade padrão

        # Determina tratamento e resultado com base nos sintomas e gravidade
        if symptoms == 'COVID-19' or symptoms == 'Tuberculose':
            severity = random.choice(['Leve', 'Médio', 'Grave'])
            if severity == 'Grave':
                treatment = 'Tratamento Intensivo'
                outcome = 'Encaminhado'
            elif severity == 'Médio':
                 treatment = random.choice(['Antibióticos Orais', 'Antibióticos Injetáveis', 'Tratamento Intensivo'])
                 outcome = 'Encaminhado' if random.random() < 0.7 else random.choice(['Melhorou', 'Não Melhorou', 'Aguardando Resultado']) # Maior chance de encaminhado
            else: # Leve
                treatment = random.choice(['Medicação Comum (analgésicos/antitérmicos/antigripais.)', 'Antibióticos Orais'])
                outcome = random.choice(['Melhorou', 'Não Melhorou', 'Aguardando Resultado'])

        elif symptoms == 'Infecção de Garganta':
             treatment = random.choice(['Antibióticos Orais', 'Medicação Comum (analgésicos/antitérmicos/antigripais.)'])
             outcome = random.choice(['Melhorou', 'Não Melhorou', 'Aguardando Resultado'])
        else: # Resfriado
             treatment = 'Medicação Comum (analgésicos/antitérmicos/antigripais.)'
             outcome = random.choice(['Melhorou', 'Não Melhorou', 'Aguardando Resultado'])

        # Introduz uma pequena chance de melhor resultado com maior classificação de atendimento (menos impactante que fatores médicos)
        if service_rating > 7 and outcome != 'Encaminhado' and random.random() < 0.2:
            outcome = 'Melhorou'


        # Introduzindo erros e inconsistências
        if random.random() < 0.05: # 5% de chance de idade incorreta
            age = random.choice([-10, 150, 'vinte e cinco'])
        if random.random() < 0.03: # 3% de chance de altura/peso incorreto
            height = random.choice([5.5, 0.80, 'alto'])
            weight = random.choice([200, 30, 'pesado'])
        if random.random() < 0.08: # 8% de chance de variações no nome
            name = name.upper() if random.random() < 0.5 else name.lower()
        if random.random() < 0.1: # 10% de chance de formatos de data diferentes
             consultation_date = consultation_date.strftime(random.choice(['%d/%m/%Y', '%Y-%m-%d', '%m-%d-%Y']))


        data.append([name, age, height, weight, gender, medical_history, consultation_date, outcome, service_rating, symptoms, severity, treatment])

    df = pd.DataFrame(data, columns=['Nome', 'Idade', 'Altura', 'Peso', 'Gênero', 'Histórico Médico', 'Data da Consulta', 'Resultado', 'Classificação do Atendimento', 'Sintomas', 'Gravidade', 'Tratamento'])

    # Converte colunas para float para lidar com potenciais outliers float
    for col in ['Idade', 'Altura', 'Peso', 'Classificação do Atendimento']:
        # Converte para numérico, tratando erros como NaN, depois para float
        df[col] = pd.to_numeric(df[col], errors='coerce').astype(float)


    # Adiciona alguns outliers
    for _ in range(int(num_records * 0.01)): # 1% de outliers
        random_row = random.randint(0, num_records - 1)
        df.loc[random_row, random.choice(['Idade', 'Altura', 'Peso', 'Classificação do Atendimento'])] = random.choice([200.0, 0.5, 300.0, -5.0, 15.0]) # Garante que outliers são floats


    # Adiciona alguns duplicados
    duplicate_rows = df.sample(frac=0.02) # 2% de duplicados
    df = pd.concat([df, duplicate_rows], ignore_index=True)

    return df

# Gera um conjunto de dados de exemplo
num_records = 10000 # Aumentado para 10000
medical_df = generate_medical_data(num_records)

display(medical_df.head())

,Nome,Idade,Altura,Peso,Gênero,Histórico Médico,Data da Consulta,Resultado,Classificação do Atendimento,Sintomas,Gravidade,Tratamento
0,Sra. Maria Cecília Pimenta,34.0,1.70,95.02,Masculino,Repellendus necessitatibus architecto.,2025-04-20,Encaminhado,6.0,Tuberculose,Grave,Tratamento Intensivo
1,Maria Clara Sales,49.0,1.55,50.49,NaN,Deleniti molestiae fugit nam aspernatur aut.,2024-05-12,Aguardando Resultado,9.0,Resfriado,NaN,Medicação Comum (analgésicos/antitérmicos/anti...
2,Elisa da Mota,50.0,1.81,86.86,Outro,Facilis dolorem ducimus veritatis perferendis ...,2024-10-27,Encaminhado,4.0,Tuberculose,Médio,Antibióticos Injetáveis
3,Zoe Guerra,74.0,1.77,53.32,Feminino,Corrupti ab eveniet unde inventore.,2024-10-21,Não Melhorou,10.0,COVID-19,Leve,Antibióticos Orais
4,Rodrigo Nascimento,79.0,1.59,62.36,NaN,Amet eaque sint rem quae error dolorem rerum.,2025-08-12,Aguardando Resultado,4.0,Resfriado,NaN,Medicação Comum (analgésicos/antitérmicos/anti...
